# Lab 8.1: nodes modification
by Domrachev Ivan, B20-Ro-01

In [1]:
from nn_from_scratch.neurons import Convolution

import numpy as np
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers

2023-11-07 13:40:05.633744: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-07 13:40:05.694734: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-07 13:40:05.695521: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-07 13:40:06.629865: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Part 1. Bathched & Biased Convolution

The convolutional layer was extended to support batches inputs and optional bias input
> Note: the solution is far from being generalized, f.e. it lacks padding, stride settings, as well as support of batches of the pictures

In [2]:
input_dim = (2, 3, 4, 5)
kernel_size = 2
output_layers = 20
conv = Convolution(input_dim, kernel_size, output_layers=output_layers)
output_dim = conv._output_dim

# Random input values (x itself, and assumed partial derivative)
x_input = np.random.random(input_dim)
dL_dy = np.random.random(output_dim)

output = conv.forward(x_input)
dL_dx = conv.backward(dL_dy)
dL_dw = conv._W_pd
bias = conv._B
dL_db = conv._B_pd

In [3]:
print(dL_dx.shape)

(2, 3, 4, 5)


As before, let's compare its performance with tensorflow implementation:

In [4]:
x_input_batched = tf.constant(
    np.moveaxis(
        x_input,
        1, -1
    ), 
    dtype=tf.float32
)

weights_reshaped = tf.constant(
    np.moveaxis(
        conv.W,
        0, -1
    ), 
    dtype=tf.float32
)

2023-11-07 13:40:08.037232: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-07 13:40:08.037964: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
x_input_batched.shape

TensorShape([2, 4, 5, 3])

In [6]:
conv_keras = layers.Conv2D(
    20, 2,
    input_shape=x_input_batched.shape[1:],
    use_bias=False,
    kernel_initializer=tf.keras.initializers.Constant(weights_reshaped)
)

In [7]:
with tf.GradientTape(persistent=True) as tape:
    tape.watch(x_input_batched)  # Watch the input tensor for gradient computation
    conv_output = conv_keras(x_input_batched)

conv_output_np = conv_output.numpy().transpose(0, 3, 1, 2).squeeze()

1. Check that layer output is correct

In [8]:
np.all((conv_output_np - output[0]) < 1)

True

2. Check that backpropogation for both input and weights is correct

In [9]:
dL_dy_keras = tf.constant(np.moveaxis(dL_dy, 1, -1), dtype=tf.float32)

dL_dx_keras = tape.gradient(
    conv_output, x_input_batched, output_gradients=dL_dy_keras
)
dL_dw_keras = tape.gradient(
    conv_output, conv_keras.trainable_variables, output_gradients=dL_dy_keras
)

dL_dx_keras_np = np.moveaxis(dL_dx_keras[0].numpy().squeeze(), -1, 0)
dL_dw_keras_np = np.moveaxis(dL_dw_keras[0].numpy().squeeze(), -1, 0)

In [1]:
dL_dx_keras_np

NameError: name 'dL_dx_keras_np' is not defined

In [16]:
np.all((dL_dx_keras_np - dL_dx[0]) < 1)

False

In [12]:
np.all((dL_dw_keras_np - np.moveaxis(dL_dw, 1, -1)) < 1e-4)

True

## Support for tensor inputs

### ReLU

It's quite trivial to add 4D tensor support to ReLU, but anyway, let's validate it's functionality:

In [13]:
input_dim = (5, 3, 10, 10)
conv = Convolution(input_dim, kernel_size, output_layers=output_layers)
output_dim = conv._output_dim

# Random input values (x itself, and assumed partial derivative)
x_input = np.random.random(input_dim)
dL_dy = np.random.random(output_dim)

output = conv.forward(x_input)
dL_dx = conv.backward(dL_dy)
dL_dw = conv._W_pd
bias = conv._B
dL_db = conv._B_pd


In [14]:
from nn_from_scratch.nodes import ReLU  
import numpy as np

input_dim = (2, 2, 2, 2)

# Random input values (x itself, and assumed partial derivative)
x_input = np.random.randint(-10, 10, size=input_dim)
dL_dy = np.random.randint(-10, 10, size=input_dim)
print(x_input)
relu = ReLU(input_dim)

# Forward call
y_value = relu.forward(x_input)

# Backpropogation
dL_dx = relu.backward(dL_dy)

y_value, dL_dx

[[[[ -9  -7]
   [  2   8]]

  [[ -2   0]
   [  4   1]]]


 [[[  7   4]
   [  9  -1]]

  [[ -6   6]
   [ -2 -10]]]]


(array([[[[0, 0],
          [2, 8]],
 
         [[0, 0],
          [4, 1]]],
 
 
        [[[7, 4],
          [9, 0]],
 
         [[0, 6],
          [0, 0]]]]),
 array([[[[ 0,  0],
          [-3, -3]],
 
         [[ 0,  0],
          [ 1, -3]]],
 
 
        [[[-1,  7],
          [ 8,  0]],
 
         [[ 0,  4],
          [ 0,  0]]]]))

As one could see, the functionality works

## Vectorization

Another handy node is Vectorizer -- it transforms everything into vector. It's very handy, since it allows to utilize existing code for CNN as well!

In [15]:
from nn_from_scratch.nodes import Vectorization  
import numpy as np

input_dim = (2, 2, 2, 2)

# Random input values (x itself, and assumed partial derivative)
x_input = np.random.randint(-10, 10, size=input_dim)

vectorize = Vectorization(input_dim)
output_dim = vectorize._output_dim
dL_dy = np.random.randint(-10, 10, size=output_dim)

# Forward call
y_value = vectorize(x_input)

# Backpropogation
dL_dx = vectorize.backward(dL_dy)

y_value.shape, dL_dx.shape

((2, 8), (2, 2, 2, 2))